# Ćwiczenia 6. Walidacja Krzyżowa

## PyTorch na następne ćwiczenia.

Proszę zainstalować pakiet [PyTorch](https://pytorch.org/) oraz torchvision na kolejne zajęcia. Jeśli używane, mając swoje środowisko aktywne użyć:

 * GPU: `conda install pytorch torchvision cudatoolkit=9.0 -c pytorch`
 * tylko CPU: `conda install pytorch torchvision cpuonly  -c pytorch`

## Klasyfikacja

Dzisiaj na zajęciach zajmiemy się problemem klasyfikacji. Podobnie do regresji liniowej jest to przykład uczenia nadzorowanego, ale zamiast przewidywać konkretną liczbę dla danej obserwacji, przewidujemy jego przynajeżność do jednej z *k* klas. Na tych zajęciach będziemy rozważać klasyfikacje binarną, czyli uczyć modele odpowiadające funkcji:

$$ f(x) = y, \quad y \in \{0,1\} $$

Poniżej ładowane są dane, do razu już podzielone na dwie części.

In [1]:
import numpy as np
from utils import get_data

X_train, X_test, y_train, y_test = get_data()

## Zadanie 1.1 (0.5 pkt.)

Używając modelu [`SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) z pakietu sklearn uzyskać 100% dokładność (mierzoną miarą [`accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html))na zbiorze treningowym. Państwa zadanie polega na dobraniu parametru `gamma`, dla ułatwienia proszę nie zmieniać pozostałych domyślnych parametrów. Zalecany przedział parametru `gamma` to wartości z przedziału [0, 1] w skali logarytmicznej.

In [2]:
from sklearn.svm import SVC

# your code here
"""wpisanie czegokolwiek powyzej 1 daje dobry wynik"""

'wpisanie czegokolwiek powyzej 1 daje dobry wynik'

In [3]:
# test
best_gamma = 1# ???
svm = SVC(gamma=best_gamma)
svm.fit(X_train, y_train)
train_acc = svm.score(X_train, y_train)
assert train_acc == 1.

## Zadanie 1.2 (0.5 pkt.)
Używając tej samej rodziny modeli znajdź tym razem taką wartość `gamma`, która daje najlepszy wynik na zbiorze **testowym**. Powinieneś(-aś) być w stanie osiągnąć wynik co najmniej `0.95` accuracy. 

In [4]:
from sklearn.svm import SVC
gamma=[ np.exp(i) for i in range(10)] #gamma>1
gamma2=[ np.exp(-i) for i in range(20)] #gamma <1
gamma=gamma+gamma2
svm_table = [ SVC(gamma=val) for val in gamma]
for svm in svm_table : svm.fit(X_train, y_train)
testy_acc=[]
for svm in svm_table: testy_acc.append(svm.score(X_test, y_test))
print( testy_acc)

[0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6153846153846154, 0.9090909090909091, 0.9300699300699301, 0.951048951048951, 0.965034965034965, 0.972027972027972, 0.951048951048951, 0.951048951048951, 0.951048951048951, 0.9440559440559441, 0.9370629370629371, 0.9370629370629371, 0.9090909090909091, 0.8811188811188811, 0.8181818181818182]


In [ ]:
# test
#best_gamma = np.exp(-10) 
best_gamma=np.exp(-10)
svm = SVC(gamma=best_gamma)
svm.fit(X_train, y_train)
test_acc = svm.score(X_test, y_test)
print(test_acc)
assert test_acc >= 0.95


0.972027972027972


### Problem.

**W poprzednim zadaniu zakładaliśmy, że podział na zbiór trenujący/testujący jest nam podany z góry, ale co jeśli go nie mamy?**

Możemy oczywiście sami arbitralnie wybrać część datasetu i uznać ją za nasz zbiór testowy, ale to mogą się z tym wiązać dodatkowe problemy: co jeśli wybrany przez nas fragment jest akurat różny od reszty datasetu, lub odwrotnie?

**Rozwiązanie:** Walidacja Krzyżowa.

1. Podziel dataset na zadaną przez parametr `k` liczbę (prawie) równych grup.
2. Dla każdego podziału, zwróć jedną z tych części jako zbiór testowy, a sumę reszty jako zbiór treningowy.
3. Po nauczeniu łącznie `k` modeli, uśrednij ich wyniki na zbiorach testowych i zwróć jako ostateczny wynik.

## Zadanie 2. (2 pkt.)

Państwa zadaniem jest zaimplementowanie walidacji krzyżowej, czyli funkcji, która dla podanego datasetu w postaci macierzy danych `X` i wektora etykiet `y` zwróci listę `k` krotek: 
  
  `(treningowe_dane, treningowe_etykiety, testowe_dane, testowe_etykiety)`
  
podziałów dokonanych przez walidację krzyżową. Następnie należy użyć modelu z poprzedniego zadania dla policzenia dokładności na zbiorze testowym dla walidacji krzyżowej.

Proszę **nie** korzystać z gotowych rozwiązań dostępnych w pakiecie sklearn.


In [2]:
from typing import List, Tuple


def cross_validation(X: np.ndarray, y: np.ndarray, k: int) -> List[Tuple[np.ndarray, np.ndarray, 
                                                                         np.ndarray, np.ndarray]]:
    p=np.random.permutation(X.shape[0])
    X=X[p]
    y=y[p]
    step=int(X.shape[0]/k)
    ret_list=[]
    for i in range(k-1):
        a=np.concatenate((X[:i*step],X[(i+1)*step:]))
        b=np.concatenate((y[:i*step],y[(i+1)*step:]))
        c=X[i*step:(i+1)*step]
        d=y[i*step:(i+1)*step]
        ret_list.append((a,b,c,d))
    ret_list.append((X[:(k-1)*step],y[:(k-1)*step],X[(k-1)*step:],y[(k-1)*step:]))   
    return ret_list

In [3]:
from checker import test_cv

test_cv(cross_validation)

In [ ]:
X, y = get_data(split=False)
def accuracy(X,y,k=5,gamma=1,C=1):
    dane=cross_validation(X,y,k)
    svm=SVC(gamma=gamma,C=C)
    acc=0
    for a,b,c,d in dane:
        svm.fit(a,b)
        #print(svm.score(c,d))
        acc=acc+svm.score(c,d)

    cv_accuracy = acc/k
    return cv_accuracy
print(accuracy(X,y,5,gamma=np.exp(-10),C=1))

0.9312154905075258


## Zadanie 3 (1 pkt.)

Mając już lepszą metodę walidacji naszego rozwiązania Państwa zadaniem jest znalezienia najlepszego zestawu hiperparametrów dla modelu z poprzednich zadań, lecz tym razem będziemy szukać również parametru `C`. Parametru C zaleca się szukać w przedziale $(0, + \infty)$ również w skali logarytmicznej.

W zadaniu należy oczywiście skorzystać z zaimplementowanej przez Państwa walidacji krzyżowej. Ponieważ dla dwóch parametrów `C` oraz `gamma` możliwych kombinacji do przetestowania robi są dość sporo dla przetestowania dużych zakresów zalecane są również inne metody przeszukiwania, takie jak:

* przeszukiwanie po kolei z jednym z parametrów ustalonym na stałą.
* przeszukiwanie losowe obu parametrów

Oczywiście jeśli zasoby na to pozwalają można szukać tzw. "grid searchem".

Powinno udać się Państwu wyciągnąć przynajmniej `0.94` accuracy na walidacji krzyżowej.

In [ ]:
from sklearn.svm import SVC

X, y = get_data(split=False)


gamma=np.exp(np.linspace(1,20,5)) 
gamma2=np.exp(-np.linspace(1,20,5)) 
gamma=np.concatenate((gamma,gamma2))
C=np.exp(-np.linspace(1,10,5)) 
C2=np.exp(np.linspace(1,10,5)) 
C=np.concatenate((C,C2))
#print(gamma,"\n",C)
datas=cross_validation(X,y,3)
best,mean=[],[]
for X_train,y_train,X_test,y_test in datas:
    best_parameters=(1,1,0) # gamma, C ,accuracy
    for g in gamma:
        for c in C:
            svm=SVC(gamma=g,C=c)
            svm.fit(X_train,y_train)
            score=svm.score(X_test,y_test)
            if score > best_parameters[2] : best_parameters=(g,c,score)      
    print(best_parameters)
    best.append(best_parameters)
    mean.append(best_parameters[2])
        #print(g,c,accuracy(X,y,5,gamma=g,C=c))
print("Srednia acuracy na teście:",np.mean(np.array(mean)) )

(2.7536449349747158e-05, 244.69193226422038, 0.9841269841269841)
(2.382369667501818e-07, 22026.465794806718, 0.9153439153439153)
(2.7536449349747158e-05, 244.69193226422038, 0.9581151832460733)
Srednia acuracy na teście: 0.9525286942389909


In [ ]:
best_gamma=2.382369667501818e-07
best_C= 22026.465794806718
test=accuracy(X,y,5,gamma=best_gamma,C=best_C)
print(test)
assert test >= 0.94

0.9508660464412676


## Zadanie 4. (3 punkty)

Załóżmy, że naszym problemem jest zdecydowanie, która rodzina modeli *SVM* najlepiej radzi sobei z naszym datasetem. Przez rodzinę rozumiemy tutaj modele SVM z różną *funkcją bazwoą* (zwaną często *funkcją jądra*). W pakiecie mamy dostępne kilka możliwości, włącznie z podawaniem swoich własnych, ale w tym zadaniu skupimy się na czterech dostępnych od ręki: `linear`, `poly`, `rbf`, `sigmoid`.

Wiemy jak znaleźć najlepszy zestaw parametrów dla danej rodziny modeli, zrobiliśmy to do tej pory dla domyślnej funkcji bazowej czyli `rbf`. Jak jednak mamy "uczciwie" porównać wyniki modeli pomiędzy sobą? Do tej pory patrzyliśmy na wyniki modelu dla datasetu testowego i to na podstawie tego wyniku wybieraliśmy najlepsze hiperparametry. Jakie mogą być z tym problemy? Overfitting?

Rozwiązanie: jeszcze jedna walidacja krzyżowa!

1. Pierwsza walidacja krzyżowa podzieli nam nasz zbiór na treningowy i testowy. Te testowe zbiory będą naszymi ostatecznymi zbiorami testowymi, na których nie będziemy w żaden sposób się uczyć czy szukać hiperparametrów. 
2. Następnie nasz zbiór treningowy podzielimy ponownie walidacją krzyżową na dwie części: faktyczny treningowy i walidacyjny. Tych dwóch podziałów będziemy używać jak poprzednio do uczenia modelu i testowania hiperparametrów.
3. Po znalezieniu najlepszego zestawu hiperparametrów nauczymy ostatecznie nasz model na sumie zbiorów treningowego i walidacyjnego i sprawdzimy jego dokładność na ostatecznym zbiorze testowym.


**Uwaga**: parametr `C` używany jest dla każdej możliwej funkcji bazowej. Proszę sprawdzić jakie parametry są używane dla jakich funkcji jądra. 
**Hint**: parametry, które mogą państwa interesować to oczywiście `kernel`, oraz `C`, `degree`, `gamma`, `coef0`.

In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import ParameterGrid

X, y = get_data(split=False)
C= [0.1, 1, 10, 100]
gamma=[0.1, 0.01, 0.0001] # dla gamma=0.001 bardzo muli
degree= [1,10]
coef0=[0.001,0.01,0.1,1,10,100]

datas=cross_validation(X,y,3)
#linear C 
wyniki=[]
for X_train,y_train,X_test,y_test in datas:
    #print("for X_train")
    splited=cross_validation(X_train,y_train,3)
    best_parameters=(1,0) #  C ,accuracy    
    for xt,yt,xv,yv in splited:  # x_train, y_train, x_validation, y_validation   
        #print("for xt")
        for c in C:
            #print("for c",c)
            svm=SVC(kernel='linear',C=c)
            svm.fit(xt,yt)
            score=svm.score(xv,yv)
            if score > best_parameters[-1] : best_parameters=(c,score)      
        #print("lokal",best_parameters)
    #print(best_parameters) #najlepsze parametry 
    c=best_parameters[0]
    svm=SVC(kernel='linear',C=c)
    svm.fit(X_train,y_train)
    wyniki.append(svm.score(X_test,y_test))
    print("wyliczone parametry:",best_parameters)
print("Dla modelu linear najlepszy wynik to",np.mean(np.array(wyniki)))

#poly C deegree coef0  # chodzi tak z ponad pół godziny
wyniki=[]
grid = {'c': C , 'd': degree, 'g': gamma,'f':coef0}
for X_train,y_train,X_test,y_test in datas:
    #print("for X_train")
    splited=cross_validation(X_train,y_train,3)
    best_parameters=(1,1,1,0) #  C ,degree,coef0,accuracy    
    for xt,yt,xv,yv in splited:  # x_train, y_train, x_validation, y_validation   
        #print("for xt")
        for zb in (ParameterGrid(grid)):
            #print(zb)
            c,d,g,f=zb['c'],zb['d'],zb['g'],zb['f']
            #print("for c",c)
            svm=SVC(kernel='poly',C=c,degree=d,gamma=g,coef0=f)
            #print("fituje")
            svm.fit(xt,yt)
            #print("zafitowałem")
            score=svm.score(xv,yv)
            if score > best_parameters[-1] : best_parameters=(c,d,g,f,score)      
        #print("lokal",best_parameters)
    #print(best_parameters) #najlepsze parametry 
    c,d,g,f=best_parameters[0],best_parameters[1],best_parameters[2],best_parameters[3]
    svm=SVC(kernel='poly',C=c,degree=d,gamma=g,coef0=f)
    svm.fit(X_train,y_train)
    wyniki.append(svm.score(X_test,y_test))
    print("wyliczone parametry:",best_parameters)
print("Dla modelu poly najlepszy wynik to",np.mean(np.array(wyniki)))

#rbf C gamma 
wyniki=[]
grid = {'c': C, 'g': gamma}
for X_train,y_train,X_test,y_test in datas:
    #print("for X_train")
    splited=cross_validation(X_train,y_train,3)
    best_parameters=(1,1,1,0) #  C ,degree,coef0,accuracy    
    for xt,yt,xv,yv in splited:  # x_train, y_train, x_validation, y_validation   
        #print("for xt")
        for zb in (ParameterGrid(grid)):
            #print(zb)
            c,g=zb['c'],zb['g']
            #print("for c",c)
            svm=SVC(kernel='rbf',C=c,gamma=g)
            #print("fituje")
            svm.fit(xt,yt)
            #print("zafitowałem")
            score=svm.score(xv,yv)
            if score > best_parameters[-1] : best_parameters=(c,g,score)      
        #print("lokal",best_parameters)
    #print(best_parameters) #najlepsze parametry 
    c,g=best_parameters[0],best_parameters[1]
    svm=SVC(kernel='rbf',C=c,gamma=g)
    svm.fit(X_train,y_train)
    wyniki.append(svm.score(X_test,y_test))
    print("wyliczone parametry:",best_parameters)
print("Dla modelu rbf najlepszy wynik to",np.mean(np.array(wyniki)))

#sigmoid C gamma coef0
gamma=[10,1,0.1, 0.01, 0.0001,0.00001] # ustalam więszky zakres bo jest szybko a dla sigmoidu nie chce wyjść
coef0=[0.001,0.01,0.1,1,10,100]
C= [0.1, 1, 10, 100,1000]
wyniki=[]
grid = {'c': C, 'g': gamma, 'f': coef0}
for X_train,y_train,X_test,y_test in datas:
    #print("for X_train")
    splited=cross_validation(X_train,y_train,3)
    best_parameters=(1,1,1,0) #  C ,degree,coef0,accuracy    
    for xt,yt,xv,yv in splited:  # x_train, y_train, x_validation, y_validation   
        #print("for xt")
        for zb in (ParameterGrid(grid)):
            #print(zb)
            c,g,f=zb['c'],zb['g'],zb['f']
            #print("for c",c)
            svm=SVC(kernel='sigmoid',C=c,gamma=g,coef0=f)
            #print("fituje")
            svm.fit(xt,yt)
            #print("zafitowałem")
            score=svm.score(xv,yv)
            if score > best_parameters[-1] : best_parameters=(c,g,score)      
        #print("lokal",best_parameters)
    #print(best_parameters) #najlepsze parametry 
    c,g=best_parameters[0],best_parameters[1]
    svm=SVC(kernel='sigmoid',C=c,gamma=g,coef0=f)
    svm.fit(X_train,y_train)
    wyniki.append(svm.score(X_test,y_test))
    print("wyliczone parametry:",best_parameters)
print("Dla modelu sigmoid najlepszy wynik to",np.mean(np.array(wyniki)))

wyliczone parametry: (100, 0.9841269841269841)
wyliczone parametry: (100, 0.9920634920634921)
wyliczone parametry: (0.1, 0.9444444444444444)
Dla modelu linear najlepszy wynik to 0.9437103520873155
wyliczone parametry: (0.1, 1, 0.01, 0.001, 0.9841269841269841)
wyliczone parametry: (0.1, 1, 0.1, 0.001, 0.9841269841269841)
wyliczone parametry: (100, 1, 0.1, 0.001, 0.9761904761904762)
Dla modelu poly najlepszy wynik to 0.9349104776678946
wyliczone parametry: (1, 0.0001, 0.9841269841269841)
wyliczone parametry: (10, 0.0001, 0.9603174603174603)
wyliczone parametry: (1, 0.0001, 0.9365079365079365)
Dla modelu rbf najlepszy wynik to 0.9366002751692105
wyliczone parametry: (0.1, 10, 0.6746031746031746)
wyliczone parametry: (0.1, 10, 0.7109375)
wyliczone parametry: (0.1, 10, 0.6349206349206349)
Dla modelu sigmoid najlepszy wynik to 0.6272842276332677
